In [3]:
%%time

# Road Trip Planner

from math import *  # bad idea, pollutes namespace!

FILENAME = 'usa_cities.csv'
# FILENAME = 'california.csv'
START = 'San Francisco, California'
END = 'Dallas, Texas'
# END = 'San Diego, California'
KM_PER_DAY = 250

# Read data file
# bad #1: reassigns input function
# bad #2: file descriptor is open but never closed.
input = open(FILENAME, encoding='utf-8').readlines()

# column headers
# city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
# bad idea, not very flexible. Need to juggle several lists in memory, keep track of indices, etc
cities = []
lat = []
lon = []
counties = []
admin_names = []  # states

for l in range(0, len(input)):
    line = input[l]  # readlines is a list. we can iterate over the list directly
    # example line
    # "California","California","40.0692","-79.9152","United States","US","USA","Pennsylvania","","6364","1840003642"

    # Divide in columns
    columns = line.replace('"', '').split(',')

    # ignore header and empty lines
    if len(columns) > 0 and columns[0] != 'city':  # bad: performance. Easier to skip first line.
        cities.append(columns[0])
        counties.append(columns[1])
        admin_names.append(columns[2])
        lat.append(eval(columns[3]))  # bad. Eval allows execution of arbitrary code.
        lon.append(eval(columns[4]))

print('Read', str(len(cities)), 'cities')  # f-strings

# Find query cities in database
start_city = START.split(',')[0].strip()  # modularize.
start_state = START.split(',')[1].strip()

end_city = END.split(',')[0].strip()
end_state = END.split(',')[1].strip()

start_idx = None
end_idx = None

for i in range(0, len(cities)):  # iterate over containers.
  if cities[i] == start_city and admin_names[i] == start_state:
    print('Found start', i)
    start_idx = i
  elif cities[i] == end_city and admin_names[i] == end_state:
    print('Found end', i)
    end_idx = i

# Do search
if start_idx != None and end_idx != None:

    # Find 'best' route
    route = [(start_city, start_state)]
    
    current_city = (start_city, start_state)
    dead_end = False  # easier to raise exceptions when things break than multiple flags
    do_search = True
    while do_search:
        city = current_city[0]
        state = current_city[1]
        
        # Get city coordinates
        for i in range(0, len(cities)):
            if cities[i] == city and admin_names[i] == state:
                cur_idx = i  # needless iteration. Use break or dictionary.

        # Get neighbors
        neighbors = []
        for j in range(len(cities)):
            
            # Haversine formula for distances between points on a sphere
            # https://en.wikipedia.org/wiki/Haversine_formula
            lat_i = lat[cur_idx] * 3.14 / 180
            lon_i = lon[cur_idx] * 3.14 / 180
            lat_j = lat[j] * 3.14 / 180
            lon_j = lon[j] * 3.14 / 180

            dlat = lat_j - lat_i
            dlon = lon_j - lon_i
            a = (sin(dlat/2)*sin(dlat/2)) + cos(lat_i) * cos(lat_j) * (sin(dlon/2)*sin(dlon/2))
            c = 2 * atan2( a**0.5, (1-a)**0.5 )
            d = 6373  * c  # R is 'a' radius of earth
            
            if d <= KM_PER_DAY:
                neighbors.append((j, cities[j], admin_names[j]))
        
        if (end_idx, end_city, end_state) in neighbors:  # end search
            pick = (end_city, end_state)
            do_search = False
        else:
            min_d = 99999999999999999999
            pick = None
            
            # Calculate closest neighbor to destination
            for i in range(0, len(neighbors)):
                nidx = neighbors[i][0]
                nname = neighbors[i][1]
                nstate = neighbors[i][2]

                lat_i = lat[nidx] * 3.14 / 180
                lon_i = lon[nidx] * 3.14 / 180
                lat_j = lat[end_idx] * 3.14 / 180
                lon_j = lon[end_idx] * 3.14 / 180

                dlat = lat_j - lat_i
                dlon = lon_j - lon_i
                a = (sin(dlat/2)*sin(dlat/2)) + cos(lat_i) * cos(lat_j) * (sin(dlon/2)*sin(dlon/2))
                c = 2 * atan2( a**0.5, (1-a)**0.5 )
                d = 6373  * c  # R is 'a' radius of earth

                if d <= min_d and (nname, nstate) not in route:  # refactor to use sorted
                    min_d = d
                    pick = (nname, nstate)

            if pick == None:
              do_search = False
              dead_end = True
                
        route.append(pick)
        current_city = pick

    # Print result
    if dead_end: print('Cannot find route with requested parameters')
    else:
        print('Proposed Route:')
        counter = 1  # enumerate is your friend
        for step in route:
            city = step[0]
            state = step[1]
            print('Stop #' + str(counter), ':', city + ',', state)
            counter += 1
else: print('Could not find requested cities')


Read 29880 cities
Found start 2644
Found start 2645
Found end 25148
Found end 25149
Proposed Route:
Stop #1 : San Francisco, California
Stop #2 : Oakhurst, California
Stop #3 : Tonopah, Nevada
Stop #4 : Panaca, Nevada
Stop #5 : Escalante, Utah
Stop #6 : Pleasant View, Colorado
Stop #7 : Canjilon, New Mexico
Stop #8 : Conchas Dam, New Mexico
Stop #9 : Tulia, Texas
Stop #10 : Oklaunion, Texas
Stop #11 : Irving, Texas
Stop #12 : Dallas, Texas
Wall time: 1.88 s
